In [3]:
import pyscf
import pyscf.tools
import sys
sys.path.append('../orbitalpartitioning')
import orbitalpartitioning
from orbitalpartitioning import *


In [4]:
molecule = """
C       -1.4481100000      5.7469900000      0.0000000000                 
C       -0.6943100000      6.9591700000      0.0000000000                 
C        0.6943100000      6.9591700000      0.0000000000                 
C        1.4481100000      5.7469900000      0.0000000000                 
C        0.7138400000      4.5865200000      0.0000000000                 
C       -0.7138400000      4.5865200000      0.0000000000                 
C       -1.4911700000      1.9118900000      0.0000000000                 
C       -0.7097300000      3.0743200000      0.0000000000                 
C        0.7097300000      3.0743200000      0.0000000000                 
C        1.4911700000      1.9118900000      0.0000000000                 
C        0.7119700000      0.7562600000      0.0000000000                 
C       -0.7119700000      0.7562600000      0.0000000000                 
H       -1.2269300000      7.9129800000      0.0000000000                 
H        1.2269300000      7.9129800000      0.0000000000                 
H        2.5397300000      5.7677500000      0.0000000000                 
H        2.5822500000      1.9120600000      0.0000000000                 
H       -2.5397300000      5.7677500000      0.0000000000                 
H       -2.5822500000      1.9120600000      0.0000000000                 
H        1.1965400000     -0.1549000000      0.0000000000                 
H       -1.1965400000     -0.1549000000      0.0000000000       
"""

basis = "cc-pvdz"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        #symmetry=   True,
        spin    =   0, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
#print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.RHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
#mf.conv_tol_grad = 1e-5
#mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
mf.analyze()
# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
print("size of MO coeff", C.shape)
ndocc = pymol.nelec[1]
nsing = pymol.nelec[0] - ndocc
nvirt = pymol.mol.nao - ndocc - nsing
print(ndocc)
print(nsing)
print(nvirt)
# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]

nbas = Cdocc.shape[0]



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/td/qpnnxwv93pq0t7bbdkh5rvzr0000gn/T/tmpt0grtw3n
max_memory 4000 MB (current use 0 MB)
Set gradient conv threshold to 0.0001
init E= -463.104025143628
  HOMO = -0.161826487461819  LUMO = -0.0129903228048151
cycle= 1 E= -458.901834973459  delta_E=  4.2  |g|= 0.566  |ddm|= 4.94
  HOMO = -0.28449150963952  LUMO = 0.0484222671051882
cycle= 2 E= -459.025610075965  delta_E= -0.124  |g|= 0.17  |ddm|= 1.03
  HOMO = -0.247246659990133  LUMO = 0.0926639277595593
cycle= 3 E= -459.034447426862  delta_E= -0.00884  |g|= 0.0967  |ddm|= 0.484
  HOMO = -0.260310199546865  LUMO = 0.0805266359440184
cycle= 4 E= -459.037157791842  delta_E= -0.00271  |g|= 0.0187  |d

### Define Fragments by AOs

In [44]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    #print(ao_idx)
    if ao[0] in (0,1,2,3,4,5):
        if ao[2] in ("2p"):
                print(ao)
                frag1.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (6,7,8,9,10,11):
        if ao[2] in ("2p"):
                frag2.append(ao_idx)
                full.append(ao_idx)


frags = [frag1, frag2]
print(frags)
print(len(frag1))
print(len(frag2))


(0, 'C', '2p', 'x')
(0, 'C', '2p', 'y')
(0, 'C', '2p', 'z')
(1, 'C', '2p', 'x')
(1, 'C', '2p', 'y')
(1, 'C', '2p', 'z')
(2, 'C', '2p', 'x')
(2, 'C', '2p', 'y')
(2, 'C', '2p', 'z')
(3, 'C', '2p', 'x')
(3, 'C', '2p', 'y')
(3, 'C', '2p', 'z')
(4, 'C', '2p', 'x')
(4, 'C', '2p', 'y')
(4, 'C', '2p', 'z')
(5, 'C', '2p', 'x')
(5, 'C', '2p', 'y')
(5, 'C', '2p', 'z')
[[3, 4, 5, 17, 18, 19, 31, 32, 33, 45, 46, 47, 59, 60, 61, 73, 74, 75], [87, 88, 89, 101, 102, 103, 115, 116, 117, 129, 130, 131, 143, 144, 145, 157, 158, 159]]
18
18


In [38]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    #print(ao_idx)
    if ao[0] in (0,1,2,3,4,5):
        if ao[2] in ("2p"):
            if ao[3] == "z":
                print(ao)
                frag1.append(ao_idx)
                full.append(ao_idx)
    if ao[0] in (6,7,8,9,10,11):
        if ao[2] in ("2p"):
            if ao[3] == "z":
                frag2.append(ao_idx)
                full.append(ao_idx)


frags = [frag1, frag2]
print(frags)


(0, 'C', '2p', 'z')
(1, 'C', '2p', 'z')
(2, 'C', '2p', 'z')
(3, 'C', '2p', 'z')
(4, 'C', '2p', 'z')
(5, 'C', '2p', 'z')
[[5, 19, 33, 47, 61, 75], [89, 103, 117, 131, 145, 159]]


### Define Projectors

In [45]:
# Define projectors
import numpy as np
import scipy
X = np.eye(nbas) 
X = scipy.linalg.sqrtm(S)
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])


### Project MOs onto all fragments

In [46]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
print(" Should be 1: ", np.linalg.det(Cact.T @ S @ Cact))

 Partition  208 orbitals into a total of   36 orbitals
            Index   Sing. Val. Space       
                0   1.21055790            2*
                1   1.12750588            2*
                2   1.06846979            2*
                3   1.02144566            2*
                4   1.00064620            2*
                5   0.94950880            2*
                6   0.93304061            2*
                7   0.89064005            2*
                8   0.88831026            2*
                9   0.79112244            0*
               10   0.75301183            0*
               11   0.74830627            0*
               12   0.74695661            0*
               13   0.73943776            0*
               14   0.73660091            0*
               15   0.72911380            2*
               16   0.72246669            0*
               17   0.71955353            0*
               18   0.71127106            0*
               19   0.70810407            0*
 

### Split active space into fragments

In [47]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    # (Of, Sf, Vf), (Oact, Sact, Vact) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
# Cfrags = canonicalize(Cfrags, F)


Cact = np.hstack(Cfrags)

print("nick: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
pyscf.tools.molden.from_mo(mf.mol, "Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])

print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 Fragment:  [3, 4, 5, 17, 18, 19, 31, 32, 33, 45, 46, 47, 59, 60, 61, 73, 74, 75]
 Partition   36 orbitals into a total of   18 orbitals
            Index   Sing. Val. Space       
                0   1.16752505            2*
                1   1.02186622            2*
                2   0.90923558            2*
                3   0.87576567            2*
                4   0.74249644            0*
                5   0.74242204            0*
                6   0.72660594            0*
                7   0.72051650            0*
                8   0.70996288            0*
                9   0.70762298            0*
               10   0.70021589            0*
               11   0.68938412            0*
               12   0.65449330            0*
               13   0.60217375            0*
               14   0.58377811            0*
               15   0.54194488            0*
               16   0.53592678            0*
               17   0.49288627            0*
        

### Extract Frontier Orbitals 

In [43]:
Cfrags = orbitalpartitioning.canonicalize(Cfrags, F)
[print(i.shape) for i in Cfrags]
env, act, vir = orbitalpartitioning.extract_frontier_orbitals(Cfrags, F, [(0,6,0), (0,6,0)])
[print(i.shape) for i in env]
[print(i.shape) for i in act]
[print(i.shape) for i in vir]
print(act)

(208, 6)
(208, 6)
(208, 0)
(208, 0)
(208, 6)
(208, 6)
(208, 0)
(208, 0)
[array([[-7.42795912e-17, -4.77932224e-16,  2.23160224e-16,
        -1.16126527e-16, -3.61871444e-16, -1.37145454e-16],
       [-9.69235817e-17,  5.19971381e-17, -1.00590031e-16,
        -2.31375757e-16, -7.70651633e-16, -4.15729581e-16],
       [ 3.29246897e-16,  2.05279203e-16,  6.81961599e-16,
         4.00179124e-15,  6.00006102e-15,  5.57785854e-16],
       ...,
       [ 5.05542206e-18,  2.34965522e-17, -1.16670187e-16,
         5.71116777e-17, -2.74024820e-16, -2.57842959e-16],
       [ 2.55081753e-17, -8.47334276e-17,  1.17033751e-16,
         2.21663889e-16, -3.61010550e-17,  2.95078205e-16],
       [-6.30794777e-04,  1.16070842e-03, -5.28268856e-04,
        -1.43642291e-03, -5.27710641e-03,  1.99404902e-03]]), array([[ 3.07294097e-18, -1.60635758e-16,  1.96684952e-16,
         2.10599723e-16, -5.02117755e-17, -1.02711493e-16],
       [-1.55787028e-16, -3.02923486e-16,  1.74250650e-16,
         3.30172150e-

### Make integrals

In [43]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)

Pa = mf.make_rdm1()
#Pb = mf.make_rdm1()[1]
np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
#np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

(208, 34)
(208, 12)
(208, 208)


In [42]:
print(S.shape)

(208, 208)
